In [2]:
import PySimpleGUI as sg
from pathlib import Path

In [4]:
sg.theme("BlueMono")

menu_layout = [
    ['File', ['Open', 'Save', '---', 'Exit']],
    ['Tools', ['Predictions', 'Graphs']]
]

layout = [  
    [sg.Menu(menu_layout)],
    [sg.Text('LIBMON', key = '-DOCNAME-')],
    [sg.Multiline(no_scrollbar = True, size = (30, 10), key = '-TEXTBOX-')],
     ]

# Create the Window
window = sg.Window('FOREsight', layout)

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
        
    if event == 'Open':
        file_path = sg.popup_get_file('Insert LIBMON file', no_window=True)
        if file_path:
            file = Path(file_path)
            print(file.read_text())
            window['-TEXTBOX-'].update(file.read_text())
            
        
        
    if event == 'Graphs':
        # ALEXIS 
        sg.popup('popup')
        
    if event == 'Predictions':
        # IULIA
        
        full_text = values['-TEXTBOX-']
        sg.popup(full_text)
    

    
    
window.close()

# Array shape: 14*5*14
1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
# New slice
0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000000000 0.000